# Import Dependencies

In [21]:
from allrank.data.dataset_loading import (
    load_libsvm_dataset,
    create_data_loaders,
)
from sklearn.datasets import load_svmlight_file, dump_svmlight_file
import numpy as np
import os
import torch
from torch import nn
from tqdm import tqdm

# Check the virtual environment

In [22]:
!conda env list

# conda environments:
#
base                     /home/admin1/anaconda3
DDOM                     /home/admin1/anaconda3/envs/DDOM
DDPM                  *  /home/admin1/anaconda3/envs/DDPM
FBO                      /home/admin1/anaconda3/envs/FBO
GenIE                    /home/admin1/anaconda3/envs/GenIE
SNAKE                    /home/admin1/anaconda3/envs/SNAKE
accelerate               /home/admin1/anaconda3/envs/accelerate
deeplearning             /home/admin1/anaconda3/envs/deeplearning
design-baselines         /home/admin1/anaconda3/envs/design-baselines
gaussian-disk            /home/admin1/anaconda3/envs/gaussian-disk
grad_env                 /home/admin1/anaconda3/envs/grad_env
latent-lang-diff         /home/admin1/anaconda3/envs/latent-lang-diff



# Set Seed

In [23]:
import random
import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
seed = 123
torch.cuda.manual_seed(seed)
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)
cudnn.deterministic = True
cudnn.benchmark = False

# Load and normalize the dataset

In [24]:
ds_path = "allrank/datasets/MSLR-toy/Fold1/"
FEATURES_WITHOUT_LOGARITHM = [5, 6, 7, 8, 9, 15, 19, 57, 58, 62, 75, 79, 85, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 121, 122, 127, 129, 130]
FEATURES_NEGATIVE = [110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 123, 124]
x_train, y_train, query_ids_train = load_svmlight_file(os.path.join(ds_path, "train.txt"), query_id=True)
x_test, y_test, query_ids_test = load_svmlight_file(os.path.join(ds_path, "test.txt"), query_id=True)
x_vali, y_vali, query_ids_vali = load_svmlight_file(os.path.join(ds_path, "vali.txt"), query_id=True)

x_train_transposed = x_train.toarray().T
x_test_transposed = x_test.toarray().T
x_vali_transposed = x_vali.toarray().T

x_train_normalized = np.zeros(x_train_transposed.shape)
x_test_normalized = np.zeros(x_test_transposed.shape)
x_vali_normalized = np.zeros(x_vali_transposed.shape)

eps_log = 1e-2
eps = 1e-6

for i, feat in enumerate(x_train_transposed):
    feature_vector_train = feat
    feature_vector_test = x_test_transposed[i, ]
    feature_vector_vali = x_vali_transposed[i, ]

    if i in FEATURES_NEGATIVE:
        feature_vector_train = (-1) * feature_vector_train
        feature_vector_test = (-1) * feature_vector_test
        feature_vector_vali = (-1) * feature_vector_vali

    if i not in FEATURES_WITHOUT_LOGARITHM:
        # log only if all values >= 0
        if np.all(feature_vector_train >= 0) & np.all(feature_vector_test >= 0) & np.all(feature_vector_vali >= 0):
            feature_vector_train = np.log(feature_vector_train + eps_log)
            feature_vector_test = np.log(feature_vector_test + eps_log)
            feature_vector_vali = np.log(feature_vector_vali + eps_log)
        else:
            print("Some values of feature no. {} are still < 0 which is why the feature won't be normalized".format(i))

    mean = np.mean(feature_vector_train)
    std = np.std(feature_vector_train)
    feature_vector_train = (feature_vector_train - mean) / (std + eps)
    feature_vector_test = (feature_vector_test - mean) / (std + eps)
    feature_vector_vali = (feature_vector_vali - mean) / (std + eps)
    x_train_normalized[i, ] = feature_vector_train
    x_test_normalized[i, ] = feature_vector_test
    x_vali_normalized[i, ] = feature_vector_vali

ds_normalized_path = "{}_normalized".format(ds_path)
os.makedirs(ds_normalized_path, exist_ok=True)

train_normalized_path = os.path.join(ds_normalized_path, "train.txt")
with open(train_normalized_path, "w"):
    dump_svmlight_file(x_train_normalized.T, y_train, train_normalized_path, query_id=query_ids_train)

test_normalized_path = os.path.join(ds_normalized_path, "test.txt")
with open(test_normalized_path, "w"):
    dump_svmlight_file(x_test_normalized.T, y_test, test_normalized_path, query_id=query_ids_test)

vali_normalized_path = os.path.join(ds_normalized_path, "vali.txt")
with open(vali_normalized_path, "w"):
    dump_svmlight_file(x_vali_normalized.T, y_vali, vali_normalized_path, query_id=query_ids_vali)

print("Dataset with normalized features saved here: {}.".format(ds_normalized_path))


Dataset with normalized features saved here: allrank/datasets/MSLR-toy/Fold1/_normalized.


# Load the normalized dataset

In [25]:
train_ds, val_ds = load_libsvm_dataset(
    input_path="allrank/datasets/MSLR-toy/Fold1/_normalized/",
    slate_length=240,
    validation_ds_role="vali",
)
print("train_ds:", train_ds.shape, train_ds)
print("val_ds:", val_ds.shape, val_ds)

n_features = train_ds.shape[-1]
assert (
    n_features == val_ds.shape[-1]
), "Last dimensions of train_ds and val_ds do not match!"

# train_dl, val_dl
train_dl, val_dl = create_data_loaders(
    train_ds,
    val_ds,
    num_workers=1,
    batch_size=64,
)
print("train_dl:", len(train_dl), "val_dl:", len(val_dl))
for xb, yb, indices in train_dl:
    print("x:", xb.shape, "y:", yb.shape, "indices:", indices.shape)
    break

train_ds: [21, 308, 136] <allrank.data.dataset_loading.LibSVMDataset object at 0x73f738093e80>
val_ds: [5, 163, 136] <allrank.data.dataset_loading.LibSVMDataset object at 0x73f643b4c910>
train_dl: 1 val_dl: 1
x: torch.Size([21, 240, 136]) y: torch.Size([21, 240]) indices: torch.Size([21, 240])


# Define Diffusion Models

## Define the denoise model

In [26]:
class MLP(nn.Module):

    def __init__(
            self,
            input_dim=137,
            index_dim=1,
            hidden_dim=2048,
            act=nn.ReLU(),
    ):
        super().__init__()
        self.input_dim = input_dim
        self.index_dim = index_dim
        self.hidden_dim = hidden_dim
        self.act = act
        self.main = nn.Sequential(
            nn.Linear(input_dim + index_dim, hidden_dim),
            act,
            nn.Linear(hidden_dim, hidden_dim),
            act,
            nn.Linear(hidden_dim, hidden_dim),
            act,
            nn.Linear(hidden_dim, input_dim),
        )

    def forward(self, features, t):
        # print(input.size(), t.size(), y.size())
        input = features.view(-1, self.input_dim)
        t = t.view(-1, self.index_dim).float()

        # forward
        # print(input.size(), t.size(), y.size())
        h = torch.cat([input, t], dim=1)  # concat
        output = self.main(h)  # forward
        return output.view(-1, self.input_dim)

## Define the diffusion process

In [27]:
class Diffusion(nn.Module):
    def __init__(self, model, num_features=137, n_times=1000, beta_minmax=[1e-4, 2e-2], device='cuda'):

        super(Diffusion, self).__init__()

        self.n_times = n_times
        self.n_features = num_features

        self.model = model

        # define linear variance schedule(betas)
        beta_1, beta_T = beta_minmax
        betas = torch.linspace(start=beta_1, end=beta_T, steps=n_times).to(device)  # follows DDPM paper
        self.sqrt_betas = torch.sqrt(betas)

        # define alpha for forward diffusion kernel
        self.alphas = 1 - betas
        self.sqrt_alphas = torch.sqrt(self.alphas)
        alpha_bars = torch.cumprod(self.alphas, dim=0)
        self.sqrt_one_minus_alpha_bars = torch.sqrt(1 - alpha_bars)
        self.sqrt_alpha_bars = torch.sqrt(alpha_bars)

        self.device = device

    def extract(self, a, t, x_shape):
        """
            from lucidrains' implementation
                https://github.com/lucidrains/denoising-diffusion-pytorch/blob/beb2f2d8dd9b4f2bd5be4719f37082fe061ee450/denoising_diffusion_pytorch/denoising_diffusion_pytorch.py#L376
        """
        b, *_ = t.shape
        out = a.gather(-1, t)
        return out.reshape(b, *((1,) * (len(x_shape) - 1)))

    def scale_to_minus_one_to_one(self, x):
        # according to the DDPMs paper, normalization seems to be crucial to train reverse process network
        return x * 2 - 1

    def reverse_scale_to_zero_to_one(self, x):
        return (x + 1) * 0.5

    def make_noisy(self, x_zeros, t):
        # perturb x_0 into x_t (i.e., take x_0 samples into forward diffusion kernels)
        epsilon = torch.randn_like(x_zeros).to(self.device)

        sqrt_alpha_bar = self.extract(self.sqrt_alpha_bars, t, x_zeros.shape)
        sqrt_one_minus_alpha_bar = self.extract(self.sqrt_one_minus_alpha_bars, t, x_zeros.shape)

        # Let's make noisy sample!: i.e., Forward process with fixed variance schedule
        #      i.e., sqrt(alpha_bar_t) * x_zero + sqrt(1-alpha_bar_t) * epsilon
        noisy_sample = x_zeros * sqrt_alpha_bar + epsilon * sqrt_one_minus_alpha_bar

        return noisy_sample.detach(), epsilon

    def forward(self, x_zeros, y):
        # x_zeros = self.scale_to_minus_one_to_one(x_zeros) # normalize to -1 to 1

        B, _ = x_zeros.shape

        # (1) randomly choose diffusion time-step
        t = torch.randint(low=0, high=self.n_times, size=(B,)).long().to(self.device)

        # (2) forward diffusion process: perturb x_zeros with fixed variance schedule
        perturbed, epsilon = self.make_noisy(torch.cat((x_zeros, y), dim=1), t)

        # (3) predict epsilon(noise) given perturbed data at diffusion-timestep t.
        pred_epsilon = self.model(perturbed, t)

        return perturbed, epsilon, pred_epsilon

    def denoise_at_t(self, x_t, timestep, t):
        B, _ = x_t.shape
        if t > 1:
            z = torch.randn_like(x_t).to(self.device)
        else:
            z = torch.zeros_like(x_t).to(self.device)

        # at inference, we use predicted noise(epsilon) to restore perturbed data sample.
        epsilon_pred = self.model(x_t, timestep)

        alpha = self.extract(self.alphas, timestep, x_t.shape)
        sqrt_alpha = self.extract(self.sqrt_alphas, timestep, x_t.shape)
        sqrt_one_minus_alpha_bar = self.extract(self.sqrt_one_minus_alpha_bars, timestep, x_t.shape)
        sqrt_beta = self.extract(self.sqrt_betas, timestep, x_t.shape)

        # denoise at time t, utilizing predicted noise
        x_t_minus_1 = 1 / sqrt_alpha * (x_t - (1 - alpha) / sqrt_one_minus_alpha_bar * epsilon_pred) + sqrt_beta * z

        return x_t_minus_1.clamp(-1., 1)

    def predict(self, x):
        # start from random noise vector, x_0 (for simplicity, x_T declared as x_t instead of x_T)
        x_t = torch.randn((x.size(0), self.n_features)).to(self.device)

        # autoregressively denoise from x_T to x_0

        # Denoise
        for t in range(self.n_times - 1, -1, -1):
            for j in range(5): ## Harmonization
                timestep = torch.tensor([t]).repeat_interleave(x.size(0), dim=0).long().to(self.device)
                x_t_1_unknown = self.denoise_at_t(x_t, timestep, t)
                if t > 0:
                    x_t_1_known, epsilon = self.make_noisy(x, timestep - 1)
                else:
                    x_t_1_known = x
                    epsilon = torch.zeros_like(x_t_1_known[:, :self.n_features]).to(self.device)
                x_t_1 = torch.cat((x_t_1_known[:, :self.n_features], x_t_1_unknown[:, self.n_features:]), dim=1)
                if j < 4 and t > 0:
                    # Add noise for one step
                    x_t = self.sqrt_alphas[t] * x_t_1 + self.sqrt_betas[t] * epsilon.to(self.device)
                else:
                    x_t = x_t_1

        # x_0 = self.reverse_scale_to_zero_to_one(x_t) # reverse normalization
        x_0 = x_t
        return x_0

# Training

In [28]:
cuda = True
DEVICE = torch.device("cuda:0" if cuda else "cpu")

In [29]:
model = MLP().to(DEVICE)
print(f"Model has {(sum(p.numel() for p in model.parameters() if p.requires_grad)):,} trainable parameters")
diffusion = Diffusion(model, device=DEVICE).to(DEVICE)

optimizer = torch.optim.Adam(diffusion.parameters(), lr=5e-5)
denoising_loss = nn.MSELoss()
epochs = 2500

print("Start training DDPMs...")
model.train()

for epoch in range(epochs):
    noise_prediction_loss = 0
    batch_idx = 1
    for xb, yb, indices in tqdm(train_dl, total=len(train_dl)):
        optimizer.zero_grad()

        xb = xb.to(DEVICE)
        yb = yb.to(DEVICE)
        xb = xb.view(-1, 136)
        yb = yb.view(-1, 1)

        noisy_input, epsilon, pred_epsilon = diffusion(xb, yb)
        loss = denoising_loss(pred_epsilon, epsilon)

        noise_prediction_loss += loss.item()

        loss.backward()
        optimizer.step()

    print("\tEpoch", epoch + 1, "complete!", "\t Denoising Loss: ", noise_prediction_loss / len(train_dl))
print("Finish!!")
# Save the model
torch.save(diffusion.state_dict(), "diffusion.pt")
print("Model saved!")

Model has 8,958,089 trainable parameters
Start training DDPMs...


100%|██████████| 1/1 [00:00<00:00,  6.08it/s]


	Epoch 1 complete! 	 Denoising Loss:  5.088084697723389


100%|██████████| 1/1 [00:00<00:00,  5.45it/s]


	Epoch 2 complete! 	 Denoising Loss:  1.9739230871200562


...

	Epoch 2498 complete! 	 Denoising Loss:  0.0966087207198143


100%|██████████| 1/1 [00:00<00:00,  5.72it/s]


	Epoch 2499 complete! 	 Denoising Loss:  0.10366932302713394


100%|██████████| 1/1 [00:00<00:00,  5.86it/s]


	Epoch 2500 complete! 	 Denoising Loss:  0.1020868718624115
Finish!!
Model saved!


# Prediction

In [30]:
# model = MLP().to(DEVICE)
# model.eval()
# diffusion = Diffusion(model).to(DEVICE)
#
# diffusion.load_state_dict(torch.load("diffusion.pt"), strict=False)
# diffusion.eval()

with torch.no_grad():
    for xb, yb, indices in val_dl:
            xb = xb.to(DEVICE)
            yb = yb.to(DEVICE)
            x = xb.view(-1, 136)[:100, :]
            y = yb.view(-1, 1)[:100, :]
            predicted = diffusion.predict(torch.cat((x, y), dim=1))[:, -1]
            print(f"Ground-truth: {y.cpu().numpy().reshape(-1)}")  # ground-truth
            print(f"Predicted: {predicted.cpu().numpy().reshape(-1)}")  # predicted
            break

Ground-truth: [ 0.  0.  1.  0.  1.  2.  1.  1.  0.  0.  0.  1.  0.  1.  0.  0.  0.  1.
  0.  1.  1.  1.  2.  4.  0.  4.  0.  0.  0.  1.  0.  0.  0.  0.  0.  1.
  1.  0.  1.  0.  0.  2.  0.  0.  1.  2.  1.  1.  2.  0.  1.  0.  1.  2.
  0.  0.  1.  0.  0.  3.  2.  1.  1.  0.  0.  1.  2.  0.  2.  0.  0.  0.
  0.  4.  1.  1.  0.  0.  1.  0.  0.  0.  0.  1.  1.  2.  0.  1.  1.  0.
  1.  1.  0. -1. -1. -1. -1. -1. -1. -1.]
Predicted: [ 0.  0.  1.  0.  1.  2.  1.  1.  0.  0.  0.  1.  0.  1.  0.  0.  0.  1.
  0.  1.  1.  1.  2.  4.  0.  4.  0.  0.  0.  1.  0.  0.  0.  0.  0.  1.
  1.  0.  1.  0.  0.  2.  0.  0.  1.  2.  1.  1.  2.  0.  1.  0.  1.  2.
  0.  0.  1.  0.  0.  3.  2.  1.  1.  0.  0.  1.  2.  0.  2.  0.  0.  0.
  0.  4.  1.  1.  0.  0.  1.  0.  0.  0.  0.  1.  1.  2.  0.  1.  1.  0.
  1.  1.  0. -1. -1. -1. -1. -1. -1. -1.]
